In [1]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json')
    # fill with rest
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [2]:
import re

code = '''
POOL_ADDRESSES_PROVIDER = IPoolAddressesProvider(0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e);
POOL = IPool(0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2);
POOL_IMPL = 0x5FAab9E1adbddaD0a08734BE8a52185Fd6558E14;
POOL_CONFIGURATOR = IPoolConfigurator(0x64b761D848206f447Fe2dd461b0c635Ec39EbB27);
POOL_CONFIGURATOR_IMPL = 0xFDA7ffA872bDc906D43079EA134ebC9a511db0c2;
ORACLE = IAaveOracle(0x54586bE62E3c3580375aE3723C145253060Ca0C2);
AAVE_PROTOCOL_DATA_PROVIDER = IPoolDataProvider(0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3);
ACL_MANAGER = IACLManager(0xc2aaCf6553D20d1e9d78E365AAba8032af9c85b0);
ACL_ADMIN = 0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A;
COLLECTOR = ICollector(0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c);
DEFAULT_INCENTIVES_CONTROLLER = 0x8164Cc65827dcFe994AB23944CBC90e0aa80bFcb;
DEFAULT_A_TOKEN_IMPL_REV_1 = 0x7EfFD7b47Bfd17e52fB7559d3f924201b9DbfF3d;
DEFAULT_VARIABLE_DEBT_TOKEN_IMPL_REV_1 = 0xaC725CB59D16C81061BDeA61041a8A5e73DA9EC6;
DEFAULT_STABLE_DEBT_TOKEN_IMPL_REV_1 = 0x15C5620dfFaC7c7366EED66C20Ad222DDbB1eD57;
EMISSION_MANAGER = 0x223d844fc4B006D67c0cDbd39371A9F73f69d974;
CAPS_PLUS_RISK_STEWARD = 0x82dcCF206Ae2Ab46E2099e663F70DeE77caE7778;
FREEZING_STEWARD = 0x2eE68ACb6A1319de1b49DC139894644E424fefD6;
DEBT_SWAP_ADAPTER = 0x8761e0370f94f68Db8EaA731f4fC581f6AD0Bd68;
DELEGATION_AWARE_A_TOKEN_IMPL_REV_1 = 0x21714092D90c7265F52fdfDae068EC11a23C6248;
CONFIG_ENGINE = 0xA3e44d830440dF5098520F62Ebec285B1198c51E;
POOL_ADDRESSES_PROVIDER_REGISTRY = 0xbaA999AC55EAce41CcAE355c77809e68Bb345170;
RATES_FACTORY = 0xcC47c4Fe1F7f29ff31A8b62197023aC8553C7896;
REPAY_WITH_COLLATERAL_ADAPTER = 0x02e7B8511831B1b02d9018215a0f8f500Ea5c6B3;
STATIC_A_TOKEN_FACTORY = 0x411D79b8cC43384FDE66CaBf9b6a17180c842511;
SWAP_COLLATERAL_ADAPTER = 0xADC0A53095A0af87F3aa29FE0715B5c28016364e;
UI_GHO_DATA_PROVIDER = 0x379c1EDD1A41218bdbFf960a9d5AD2818Bf61aE8;
UI_INCENTIVE_DATA_PROVIDER = 0x162A7AC02f547ad796CA549f757e2b8d1D9b10a6;
UI_POOL_DATA_PROVIDER = 0x91c0eA31b49B69Ea18607702c5d9aC360bf3dE7d;
WALLET_BALANCE_PROVIDER = 0xC7be5307ba715ce89b152f3Df0658295b3dbA8E2;
WETH_GATEWAY = 0x893411580e590D62dDBca8a703d61Cc4A8c7b2b9;
WITHDRAW_SWAP_ADAPTER = 0x78F8Bd884C3D738B74B420540659c82f392820e0;
SAVINGS_DAI_TOKEN_WRAPPER = 0xE28E2c8d240dd5eBd0adcab86fbD79df7a052034;
WRAPPED_TOKEN_GATEWAY = 0xD322A49006FC828F9B5B37Ab215F99B4E5caB19C;
}
'''

# Split the string into lines
lines = code.split('\n')

known_addresses = {}
for line in lines:
    # Splitting the line into variable name and the rest
    parts = line.split('=')
    if len(parts) == 2:
        variable_name = parts[0].strip()
        # Extracting the Ethereum address
        ethereum_address = re.search(r'0x[a-fA-F0-9]{40}', parts[1])
        if ethereum_address:
            known_addresses[ethereum_address.group()] = variable_name
            
known_addresses

{'0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e': 'POOL_ADDRESSES_PROVIDER',
 '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2': 'POOL',
 '0x5FAab9E1adbddaD0a08734BE8a52185Fd6558E14': 'POOL_IMPL',
 '0x64b761D848206f447Fe2dd461b0c635Ec39EbB27': 'POOL_CONFIGURATOR',
 '0xFDA7ffA872bDc906D43079EA134ebC9a511db0c2': 'POOL_CONFIGURATOR_IMPL',
 '0x54586bE62E3c3580375aE3723C145253060Ca0C2': 'ORACLE',
 '0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3': 'AAVE_PROTOCOL_DATA_PROVIDER',
 '0xc2aaCf6553D20d1e9d78E365AAba8032af9c85b0': 'ACL_MANAGER',
 '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A': 'ACL_ADMIN',
 '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c': 'COLLECTOR',
 '0x8164Cc65827dcFe994AB23944CBC90e0aa80bFcb': 'DEFAULT_INCENTIVES_CONTROLLER',
 '0x7EfFD7b47Bfd17e52fB7559d3f924201b9DbfF3d': 'DEFAULT_A_TOKEN_IMPL_REV_1',
 '0xaC725CB59D16C81061BDeA61041a8A5e73DA9EC6': 'DEFAULT_VARIABLE_DEBT_TOKEN_IMPL_REV_1',
 '0x15C5620dfFaC7c7366EED66C20Ad222DDbB1eD57': 'DEFAULT_STABLE_DEBT_TOKEN_IMPL_REV_1',
 '0x223d844fc4B006D6

In [3]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')



# Lists transactions of supplying rETH to specified markets
def supply_transactions(markets = ['AAVE'], timeframe = 30, token = 'rETH'):
    """
    Returns all "Supply" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: supply_transactions(AAVE, 90, rETH) returns all supply transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique supply transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for supply_lists of all markets
    all_market_supply_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Start processing supply API calls for each market
    for market in markets:
        # Make sure the requested market is supported
        if market in contract_address_abi_map.keys():
            # Set up contract address and ABI for each market
            contract_address, abi_filepath = contract_address_abi_map[market]
    
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)

            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
            
            # Here we will need some sort of if/else or switching logic to process unique calls for each market
            # In the meantime, we simply check for the only supported market, 'AAVE'
            if market == 'AAVE':
                supplies = smart_contract.events.Supply().get_logs(fromBlock=st_block,
                                                           toBlock='latest',
                                                           argument_filters={'reserve': token_address})
                # Consider simplifying supply_list.. don't know what information future markets will return, 
                # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
                supply_list = [
                    {
                        'Reserve': event['args']['reserve'],
                        'User (Wallet Address)': event['args']['user'],
                        'Amount (Wei)': event['args']['amount'],
                        'Amount (rETH)': event['args']['amount']/10**18,
                        'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                        'LogIndex': event['logIndex'],
                        'TransactionIndex': event['transactionIndex'],
                        'TransactionHash': event['transactionHash'].hex(),
                        'BlockHash': event['blockHash'].hex(),
                        'BlockNumber': event['blockNumber']
                    }
                    for event in supplies
                ]
                # Append each markets supply list results to a single all markets list
                all_market_supply_list.append(supply_list)
        # The market wasn't in our dictionary of supported markets
        else:
            print("UNSUPPORTED MARKET ENTERED")
            
    # Push withdrawls into dataframe
    recent_supplies = pd.DataFrame(all_market_supply_list[0])
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_supplies

"""
The above and below functions are extremely similar, the only difference being the API call they use, 
One function could have been used with some flow control and an additional parameter (User passes in "Borrow" or "Supply"
and the program runs accordingly); however, without knowing how future markets will behave, what their API calls will
exactly look like, and what they will return, it was decided to keep the two separate in case more complex logic is 
needed in handling either case specifically. 
"""


# Lists transactions of borrowing rETH to specified markets
def borrow_transactions(markets = ['AAVE'], timeframe = 60, token = 'rETH'):
    """
    Returns all "Borrow" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: borrow_transactions(AAVE, 90, rETH) returns all borrow transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique borrow transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for borrow_lists of all markets
    all_market_borrow_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Start processing supply API calls for each market
    for market in markets:
        # Make sure the requested market is supported
        if market in contract_address_abi_map.keys():
            # Set up contract address and ABI for each market
            contract_address, abi_filepath = contract_address_abi_map[market]
    
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)

            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
            
            # Here we will need some sort of if/else or switching logic to process unique calls for each market
            # In the meantime, we simply check for the only supported market, 'AAVE'
            if market == 'AAVE':
                borrows = smart_contract.events.Borrow().get_logs(fromBlock=st_block,
                                                           toBlock='latest',
                                                           argument_filters={'reserve': token_address})
                # Consider simplifying borrow_list.. don't know what information future markets will return, 
                # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
                borrow_list = [
                    {
                        'Reserve': event['args']['reserve'],
                        'User (Wallet Address)': event['args']['user'],
                        'Amount (Wei)': event['args']['amount'],
                        'Amount (rETH)': event['args']['amount']/10**18,
                        'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                        'LogIndex': event['logIndex'],
                        'TransactionIndex': event['transactionIndex'],
                        'TransactionHash': event['transactionHash'].hex(),
                        'BlockHash': event['blockHash'].hex(),
                        'BlockNumber': event['blockNumber']
                    }
                    for event in borrows
                ]
                # Append each markets supply list results to a single all markets list
                all_market_borrow_list.append(borrow_list)
        # The market wasn't in our dictionary of supported markets
        else:
            print("UNSUPPORTED MARKET ENTERED")
            
    # Push withdrawls into dataframe
    recent_borrows = pd.DataFrame(all_market_borrow_list[0])
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_borrows
    

In [4]:
recent = supply_transactions()
recent

,Reserve,User (Wallet Address),Amount (Wei),Amount (rETH),Timestamp,LogIndex,TransactionIndex,TransactionHash,BlockHash,BlockNumber
0,0xae78736Cd615f374D3085123A210448E74Fc6393,0x02cE0DDcBf374960FF78A8a42603ec8aA01baA0f,94757835848880563985,94.757836,2024-02-24 21:48:11,214,98,0x81c260954f54d1417b405f0a5fdcf357212e142717b1...,0x81e6ca676cd30db6b645b581968710516aeb8e8a8c48...,19300225
1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xaf686d9F2B46C8492b0E4CD0a92352e7e72Ccb97,40000000000000000000,40.000000,2024-02-24 21:58:23,87,21,0x0522d9f165cdf5be4f94e1c5a484f5d956f8519b03f8...,0x44d77a88cbff01b4538bf97e2636c3c4545f5dbf129f...,19300276
2,0xae78736Cd615f374D3085123A210448E74Fc6393,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,200270907491280273707,200.270907,2024-02-25 02:24:35,177,114,0xa2a3e14d9945bc0a52a47b1588af1c78405ffa1e31d6...,0x18776e067ac8eb405be9b2589c4ad63a1e3718a02d54...,19301598
3,0xae78736Cd615f374D3085123A210448E74Fc6393,0xad1A74a31b00ED0403Bb7d8B11130E30Ae15853C,1001323215490631450,1.001323,2024-02-25 12:39:47,154,76,0x474d1761687ea9be1e1049490dfc90ee7d65dd51c16a...,0x4cc76d9bf4b8b5512b5721573f27f9da72d8412b17dc...,19304653
4,0xae78736Cd615f374D3085123A210448E74Fc6393,0xADC0A53095A0af87F3aa29FE0715B5c28016364e,9250178903762549765,9.250179,2024-02-27 06:53:11,211,62,0xbb227628963595a77a3b1ba74bf611524fdb30d1e1da...,0x26ab9b8fec1ddc2ff634971a40c10276a235cf0ea896...,19317222
...,...,...,...,...,...,...,...,...,...,...
81,0xae78736Cd615f374D3085123A210448E74Fc6393,0xE6e3E0896a3D99Ff0e034c21ac05DBdbd54Ff56C,4910773026393419317,4.910773,2024-03-23 16:48:59,261,83,0x5cf2bfe3500ad3090f1b215fd18d2e6b850ce62fbb43...,0x1056c3e2515aaf779555adf1741c7c6a92d7badb54f9...,19498593
82,0xae78736Cd615f374D3085123A210448E74Fc6393,0x02e7B8511831B1b02d9018215a0f8f500Ea5c6B3,1979914855736984,0.001980,2024-03-23 18:12:47,295,115,0x20a0c27f7bf5bb6d5341fba05bd5edfa039e1732db88...,0xa1c7c252c4fddf3270d6476e8f56e7b3f02d88492a8b...,19499007
83,0xae78736Cd615f374D3085123A210448E74Fc6393,0x707076fb70def4b49a4765A593E86cd65Cf2feF9,1360000000000000000,1.360000,2024-03-23 20:31:23,576,329,0x52db71d7a9bb5beb3987217a0f28e3c3cde683e90e4e...,0x0702586c652026ff8bb789fbe7b193465db1912a9011...,19499688
84,0xae78736Cd615f374D3085123A210448E74Fc6393,0xdE13a331adF3B9b6F32017BC63dFf2f61A926a5a,2009999962138763382,2.010000,2024-03-24 10:23:11,197,107,0x4784933f5a5e853471386c5a8726e2d20c9160555abf...,0x7e114caee438be9d70c0e4fffa39fee2ad339d10b93d...,19503785


In [ ]:
unique_users = list(set(recent['User (Wallet Address)']))
unique_users

['0x7B7736a2C07C4332FfaD45a039d2117aE15e3f66',
 '0xE6e3E0896a3D99Ff0e034c21ac05DBdbd54Ff56C',
 '0xF821ec616fb20b8C53C043afe1122e5d4165A335',
 '0xBA8DD0ca7d57d89CD0a365251a248685f4245336',
 '0xaf686d9F2B46C8492b0E4CD0a92352e7e72Ccb97',
 '0x707076fb70def4b49a4765A593E86cd65Cf2feF9',
 '0x1E0cABeF79e70DDaca8859e9A33b306DE1a368f9',
 '0x1aEcea38B8626EEB3748234343cff427268Dd487',
 '0x6F3a308b0811bD282EA579831cd6615A7D64AE2E',
 '0x74B70c07d64D5d4EcbEA925EC66c2eA0D5B6a67f',
 '0xf14AF731edb271f24B6C93b9a3fC1a412F0Ae847',
 '0xd14Abc148A4A4dcEE1dCcd4ce5320E6A11cFe757',
 '0x4a60cE62F425c8a1c0B07083552369B011963e6B',
 '0x4d41f889ef14B2FE1A53561EC5993880429bB4d8',
 '0xc5F670Cc55ed46815118428261025919cB4e1a63',
 '0xD831409a48d8b21d59245C12B1540A0820446203',
 '0x777fB5B9c097fFD6306dBe8D0134b5CFC17C878b',
 '0xdE13a331adF3B9b6F32017BC63dFf2f61A926a5a',
 '0xE10db6C3c86F31B4448bB186C19F25AC09250b05',
 '0xf9231D28B34CD77A08542f73ca87c4411B1b8B56',
 '0x02cE0DDcBf374960FF78A8a42603ec8aA01baA0f',
 '0x8A03F11bc

In [ ]:
for wallet in unique_users:
    if wallet in known_addresses.keys():
        unique_users.remove(wallet)
unique_users

['0x7B7736a2C07C4332FfaD45a039d2117aE15e3f66',
 '0xE6e3E0896a3D99Ff0e034c21ac05DBdbd54Ff56C',
 '0xF821ec616fb20b8C53C043afe1122e5d4165A335',
 '0xBA8DD0ca7d57d89CD0a365251a248685f4245336',
 '0xaf686d9F2B46C8492b0E4CD0a92352e7e72Ccb97',
 '0x707076fb70def4b49a4765A593E86cd65Cf2feF9',
 '0x1E0cABeF79e70DDaca8859e9A33b306DE1a368f9',
 '0x1aEcea38B8626EEB3748234343cff427268Dd487',
 '0x6F3a308b0811bD282EA579831cd6615A7D64AE2E',
 '0x74B70c07d64D5d4EcbEA925EC66c2eA0D5B6a67f',
 '0xf14AF731edb271f24B6C93b9a3fC1a412F0Ae847',
 '0xd14Abc148A4A4dcEE1dCcd4ce5320E6A11cFe757',
 '0x4a60cE62F425c8a1c0B07083552369B011963e6B',
 '0x4d41f889ef14B2FE1A53561EC5993880429bB4d8',
 '0xc5F670Cc55ed46815118428261025919cB4e1a63',
 '0xD831409a48d8b21d59245C12B1540A0820446203',
 '0x777fB5B9c097fFD6306dBe8D0134b5CFC17C878b',
 '0xdE13a331adF3B9b6F32017BC63dFf2f61A926a5a',
 '0xE10db6C3c86F31B4448bB186C19F25AC09250b05',
 '0xf9231D28B34CD77A08542f73ca87c4411B1b8B56',
 '0x02cE0DDcBf374960FF78A8a42603ec8aA01baA0f',
 '0x8A03F11bc

In [22]:
# ETH Mainnet Market Pool Address - For WalletBalanceProvider
contract_addr = '0xC7be5307ba715ce89b152f3Df0658295b3dbA8E2'

# Load ABI for WalletBalanceProvider
with open("./WalletBalanceProvider_ABI.json") as f:
    WBP_abi = json.load(f)
    
# Load Contract
aave_pool_contract = w3.eth.contract(address=contract_addr, abi=WBP_abi)


In [23]:
positive_balances = []

# The address for the token we are trying to see the balances for, in this case the receipt token of rETH - "aethrETH"
receipt_tokens = {
    'rETH': ('0xCc9EE9483f662091a1de4795249E24aC0aC2630f', 'aethrETH'),
    'USDT': ('0x71fc860F7D3A592A4a98740e39dB31d25db65ae8', 'aUSDT'),
    'USDC': ('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c', 'aethUSDC'),
    'DAI': ('0x4C612E3B15b96Ff9A6faED838F8d07d479a8dD4c', 'aethsDAI'),
    'WBTC': ('0xFC4B8ED459e00e5400be803A9BB3954234FD50e3', 'aWBTC'),
    'WETH': ('0x030bA81f1c18d280636F32af80b9AAd02Cf0854e', 'aWETH')
    }
    

# Iterate through each wallet address and get the balance for rETH token
for wallet_address in unique_users:
    # Output from balanceOf is in wei, convert to true value using "/10**18"
    rETH_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['rETH'][0]).call())/ 10**18
    USDT_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['USDT'][0]).call())/ 10**18
    USDC_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['USDC'][0]).call())/ 10**18
    DAI_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['DAI'][0]).call())/ 10**18
    WBTC_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['WBTC'][0]).call())/ 10**18
    WETH_receipt_balance = (aave_pool_contract.functions.balanceOf(wallet_address, receipt_tokens['WETH'][0]).call())/ 10**18
    
    # If the user currently has a zero balance of the rETH receipt token --> no way they can be leveraged 
    if rETH_receipt_balance > 0:
        net_asset_borrow = USDC_receipt_balance + DAI_receipt_balance + WBTC_receipt_balance + WETH_receipt_balance
        formatted_log = {
                'User (Wallet Address)': wallet_address,
                'rETH Balance': rETH_receipt_balance,
                'USDT Balance': USDT_receipt_balance,
                'USDC Balance': USDC_receipt_balance,
                'DAI Balance': DAI_receipt_balance,
                'WBTC Balance': WBTC_receipt_balance,
                'WETH Balance': WETH_receipt_balance,
            }
        positive_balances.append(formatted_log)
        
sus_wallets = pd.DataFrame(positive_balances)
sus_wallets


,User (Wallet Address),rETH Balance,USDT Balance,USDC Balance,DAI Balance,WBTC Balance,WETH Balance
0,0x7B7736a2C07C4332FfaD45a039d2117aE15e3f66,11.184083,0.0,0.000000e+00,0.000000,0.0,0.0
1,0xE6e3E0896a3D99Ff0e034c21ac05DBdbd54Ff56C,4.910774,0.0,0.000000e+00,0.000000,0.0,0.0
2,0xBA8DD0ca7d57d89CD0a365251a248685f4245336,2.148174,0.0,6.207346e-08,0.000000,0.0,0.0
3,0x707076fb70def4b49a4765A593E86cd65Cf2feF9,1.360000,0.0,0.000000e+00,0.000000,0.0,0.0
4,0x1E0cABeF79e70DDaca8859e9A33b306DE1a368f9,45.468819,0.0,0.000000e+00,0.000000,0.0,0.0
5,0x1aEcea38B8626EEB3748234343cff427268Dd487,0.000070,0.0,3.000000e-17,0.000000,0.0,0.0
6,0x6F3a308b0811bD282EA579831cd6615A7D64AE2E,2.000000,0.0,0.000000e+00,0.000000,0.0,0.0
7,0x74B70c07d64D5d4EcbEA925EC66c2eA0D5B6a67f,23.000080,0.0,0.000000e+00,0.000000,0.0,0.0
8,0xd14Abc148A4A4dcEE1dCcd4ce5320E6A11cFe757,205.000122,0.0,0.000000e+00,0.000000,0.0,0.0
9,0x4d41f889ef14B2FE1A53561EC5993880429bB4d8,110.524662,0.0,0.000000e+00,0.000000,0.0,0.0


In [39]:
import requests

# Fetch the current token price in USD from an external data source (replace this with your own price API)
rETH_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['rETH'][0]))
rETH_price_data = rETH_price_response.json()

USDT_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['USDT'][0]))
USDT_price_data = USDT_price_response.json()

USDC_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['USDC'][0]))
USDC_price_data = USDC_price_response.json()

DAI_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['DAI'][0]))
DAI_price_data = DAI_price_response.json()

WBTC_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['WBTC'][0]))
WBTC_price_data = WBTC_price_response.json()

WETH_price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens['WETH'][0]))
WETH_price_data = WETH_price_response.json()

WETH_price_data


{'status': {'error_code': 429,
  'error_message': "You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}

In [38]:
USD_balances = []

# Function was made due to an error in processing "WETH". Due to the coingecko returning a slighly different contract address value,
# ... A solution had to be made that assumed no knowledge of the dict key value (Couldn't use "WETH_price_data[receipt_tokens['WETH'][0].lower()]", because the key 
# ... in "WETH_price_data" doesn't match our entry in "receipt_tokens['WETH'][0]"
def process_inner_dict(outer_dict):
    for inner_dict in outer_dict.values():
        if 'usd' in inner_dict:
            usd_value = inner_dict['usd']
            return usd_value
        
# Extract the token price in USD from the response
rETH_price_usd = process_inner_dict(rETH_price_data)
USDT_price_usd = USDT_price_data[receipt_tokens['USDT'][0].lower()]["usd"]
USDC_price_usd = USDC_price_data[receipt_tokens['USDC'][0].lower()]["usd"]
DAI_price_usd = DAI_price_data[receipt_tokens['DAI'][0].lower()]["usd"]
WBTC_price_usd = WBTC_price_data[receipt_tokens['WBTC'][0].lower()]["usd"]
WETH_price_usd = process_inner_dict(WETH_price_data)

# Calculate the USD value of the token balance
usd_value = positive_balances[0]['rETH Balance'] * rETH_price_usd

for index, user in enumerate(positive_balances):
    rETH_USD_balance = positive_balances[index]['rETH Balance'] * rETH_price_usd
    USDT_USD_balance = positive_balances[index]['USDT Balance'] * USDT_price_usd
    USDC_USD_balance = positive_balances[index]['USDC Balance'] * USDC_price_usd
    DAI_USD_balance = positive_balances[index]['DAI Balance'] * DAI_price_usd
    WBTC_USD_balance = positive_balances[index]['WBTC Balance'] * WBTC_price_usd
    #WETH_USD_balance = positive_balances[index]['WETH Balance'] * WETH_price_usd
    
    net_asset_borrow = USDC_USD_balance + USDC_USD_balance + DAI_USD_balance + WBTC_USD_balance #+ WETH_USD_balance
    formatted_log = {
            'User (Wallet Address)': wallet_address,
            'rETH Balance': rETH_USD_balance,
            'USDT Balance': USDT_USD_balance,
            'USDC Balance': USDC_USD_balance,
            'DAI Balance': DAI_USD_balance,
            'WBTC Balance': WBTC_USD_balance,
            #'WETH Balance': WETH_USD_balance,
            'Net Asset Borrow': net_asset_borrow
        }
    USD_balances.append(formatted_log)
        
sus_wallets_usd = pd.DataFrame(USD_balances)
sus_wallets_usd

,User (Wallet Address),rETH Balance,USDT Balance,USDC Balance,DAI Balance,WBTC Balance,Net Asset Borrow
0,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,4.148926e+04,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
1,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,1.821735e+04,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
2,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,7.969016e+03,0.0,6.207346e-08,0.000000,0.0,1.241469e-07
3,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,5.045152e+03,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
4,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,1.686743e+05,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
5,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,2.580968e-01,0.0,3.000000e-17,0.000000,0.0,6.000000e-17
6,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,7.419340e+03,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
7,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,8.532271e+04,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
8,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,7.604828e+05,0.0,0.000000e+00,0.000000,0.0,0.000000e+00
9,0x77FBd7A41cfAB09b273c12Ecf3D217688fade9c5,4.100100e+05,0.0,0.000000e+00,0.000000,0.0,0.000000e+00


In [ ]:
# TO DO
# At this point, we are able to identify leveraged users based on the net total of assets borrowed (Balance of ll assets besides rETH added up)
    # and the notional amount of rETH deposited (rETH Balance)
# From this point onward, this information can be catalogued daily to be used for historical lookback. In this way, today would be day 1, and the furthest we would be able to "look back"
# Conducting historical lookback for previous timeperiods would require a "balanceOf" function which, instead of returning the current balance, returns the balance of a given wallet 
    # at some point (timestamp) in the past. This may be possible, but hasn't yet been explored. 
# Note that we also believed it should be possible to simply acquire the current holders of a particular token using some web3 command like "holders", or some mapping ("balances")
    # This method is being explored and should replace the need for our borrow_transactions function, which currently makes the incorrect assumption that anyone who is leveraged
    # must have submitted a borrow transaction within the last x number of days (30 days in the present implementation).